In [1]:
import os
import json
#from cltk import NLP 
#from cltk.core.data_types import Word, Doc, Sentence
# from collections import Counter
from tqdm import tqdm
# import pickle
import pandas as pd
from utils import word_to_simple_identifier, sentence_to_string_repr

In [2]:
df_sents = pd.read_parquet("../data/sentences.parquet").drop_duplicates(subset=["sentence_txt"])
df_words = pd.read_parquet("../data/word_to_lemma.parquet")

In [3]:
text_counts = df_words.groupby("text").sum("count")
lemma_counts = df_words.groupby("lemma").sum("count")

In [4]:
def wordlist_matches(row: dict, text: str, lemma: str, pos: str):
    wl = row["sentence_obj"]
    return any(w["string"] == text and w["lemma"] == lemma and w["pos"] == pos for w in wl)
def find_matching_sentences(word_row: dict):
    return df_sents[df_sents.apply(lambda row: wordlist_matches(row, word_row['text'], word_row['lemma'], word_row['pos']), axis=1)].sort_values(by=["len_words", "len_chars"])
def filter_matching_sentences(matching_sents: pd.DataFrame):
    return matching_sents[matching_sents["len_words"] > 3].head(3)

In [5]:
idx = 1
matching_word = df_words.iloc[idx]
print(matching_word["index"])
matching_sents = find_matching_sentences(matching_word)
matching_sents = filter_matching_sentences(matching_sents)

δὲ [lemma: δέ] (adverb)


In [6]:
matching_sents

,metadata,sentence_obj,sentence_txt,len_words,len_chars
17189,"{'author': 'xenophon', 'edition': 'Xenophon, c...","[{'category': {'F': 'pos', 'N': 'pos', 'V': 'n...",ὁ δὲ ἔφη·,4,9
18834,"{'author': 'xenophon', 'edition': 'Xenophon, c...","[{'category': {'F': 'pos', 'N': 'pos', 'V': 'n...",ὁ δὲ εἶπε·,4,10
8478,"{'author': 'herodotus', 'edition': '', 'file':...","[{'category': {'F': 'pos', 'N': 'pos', 'V': 'n...",ὁ δὲ οὐδὲν.,4,11


In [7]:
matching_sents.to_dict(orient="records")[2]

{'metadata': {'author': 'herodotus',
  'edition': '',
  'file': 'herodotus__the-histories__grc.json',
  'language': 'unknown',
  'sentence_idx': '991_11',
  'title': 'histories',
  'urn': ''},
 'sentence_obj': array([{'category': {'F': 'pos', 'N': 'pos', 'V': 'neg'}, 'features': {'Aspect': None, 'Case': 'nominative', 'Definiteness': None, 'Degree': None, 'Gender': 'masculine', 'Mood': None, 'Number': 'singular', 'Person': None, 'Polarity': None, 'Possessive': None, 'PronominalType': None, 'Reflexive': None, 'Tense': None, 'VerbForm': None, 'Voice': None}, 'governor': 1209, 'index_token': 1207, 'lemma': 'ὁ', 'pos': 'determiner', 'string': 'ὁ', 'upos': 'DET'},
        {'category': {'F': 'neg', 'N': 'pos', 'V': 'pos'}, 'features': {'Aspect': None, 'Case': None, 'Definiteness': None, 'Degree': None, 'Gender': None, 'Mood': None, 'Number': None, 'Person': None, 'Polarity': None, 'Possessive': None, 'PronominalType': None, 'Reflexive': None, 'Tense': None, 'VerbForm': None, 'Voice': None}, '

In [8]:
for sentence in matching_sents.to_dict(orient="records"):
    s_prompt = sentence_to_string_repr(sentence)
    print(s_prompt)
    print()

--- Phrase:
ὁ δὲ ἔφη·
--- Source:
Cyropaedia by xenophon
--- Words:
ὁ [lemma: ὁ] (determiner) | singular.masculine.nominative
δὲ [lemma: δέ] (adverb)
ἔφη [lemma: φημί] (verb) | finite; past.active.indicative.third.singular
· [lemma: ·] (punctuation)

--- Phrase:
ὁ δὲ εἶπε·
--- Source:
Cyropaedia by xenophon
--- Words:
ὁ [lemma: ὁ] (determiner) | singular.masculine.nominative
δὲ [lemma: δέ] (adverb)
εἶπε [lemma: εἶπον] (verb) | finite; past.active.indicative.third.singular
· [lemma: ·] (punctuation)

--- Phrase:
ὁ δὲ οὐδὲν.
--- Source:
histories by herodotus
--- Words:
ὁ [lemma: ὁ] (determiner) | singular.masculine.nominative
δὲ [lemma: δέ] (adverb)
οὐδὲν [lemma: οὐδείς] (adjective) | singular.neuter.accusative
. [lemma: .] (punctuation)



In [4]:
df_words.pos.value_counts()

pos
verb                         38347
noun                         29651
adjective                    15811
adverb                        2197
proper_noun                   1823
pronoun                       1450
determiner                     462
adposition                     267
auxiliary                      214
numeral                        186
particle                       127
subordinating_conjunction      121
coordinating_conjunction        76
interjection                    62
other                           16
punctuation                      2
Name: count, dtype: int64

,index,text,count,lemma,pos,Aspect,Case,Definiteness,Degree,Gender,Mood,Number,Person,Polarity,Possessive,PronominalType,Reflexive,Tense,VerbForm,Voice
2,"καὶ_coordinating_conjunction_{'Aspect': None, ...",καὶ,76990,καί,coordinating_conjunction,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,"δὲ_adverb_{'Aspect': None, 'Case': None, 'Defi...",δὲ,26947,δέ,adverb,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,"καὶ_adverb_{'Aspect': None, 'Case': None, 'Def...",καὶ,21000,καί,adverb,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,"ἐν_adposition_{'Aspect': None, 'Case': None, '...",ἐν,19662,ἐν,adposition,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
7,"μὲν_adverb_{'Aspect': None, 'Case': None, 'Def...",μὲν,17206,μέν,adverb,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91139,"Προιτίδας_noun_{'Aspect': None, 'Case': 'accus...",Προιτίδας,2,Προιτίς,noun,None,accusative,None,None,feminine,None,plural,None,None,None,None,None,None,None,None
91140,"γοεδνὰ_noun_{'Aspect': None, 'Case': 'nominati...",γοεδνὰ,2,γοεδνὰ,noun,None,nominative,None,None,feminine,None,singular,None,None,None,None,None,None,None,None
91141,"ἕρμα_noun_{'Aspect': None, 'Case': 'nominative...",ἕρμα,2,ἕρμα,noun,None,nominative,None,None,neuter,None,singular,None,None,None,None,None,None,None,None
91142,"διαθεῖναι_verb_{'Aspect': 'perfective', 'Case'...",διαθεῖναι,2,διατίθημι,verb,perfective,None,None,None,None,None,None,None,None,None,None,None,past,infinitive,active


In [45]:
df_words[df_words["pos"] == "verb"]

,index,text,count,lemma,pos,Aspect,Case,Definiteness,Degree,Gender,Mood,Number,Person,Polarity,Possessive,PronominalType,Reflexive,Tense,VerbForm,Voice
61,"ἔφη_verb_{'Aspect': 'imperfective', 'Case': No...",ἔφη,3506,φημί,verb,imperfective,None,None,None,None,indicative,singular,third,None,None,None,None,past,finite,active
99,"ΣΩ_verb_{'Aspect': None, 'Case': None, 'Defini...",ΣΩ,2082,ΣΩ,verb,None,None,None,None,None,imperative,singular,second,None,None,None,None,present,finite,active
106,"ἀλλʼ_verb_{'Aspect': None, 'Case': None, 'Defi...",ἀλλʼ,1820,ἀλλʼ,verb,None,None,None,None,None,indicative,singular,second,None,None,None,None,present,finite,active
151,"ἔχειν_verb_{'Aspect': None, 'Case': None, 'Def...",ἔχειν,1201,ἔχω,verb,None,None,None,None,None,None,None,None,None,None,None,None,present,infinitive,active
152,"δοκεῖ_verb_{'Aspect': None, 'Case': None, 'Def...",δοκεῖ,1187,δοκέω,verb,None,None,None,None,None,indicative,singular,third,None,None,None,None,present,finite,active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91131,"κατετόξευσεν_verb_{'Aspect': 'perfective', 'Ca...",κατετόξευσεν,2,κατετόξευσεν,verb,perfective,None,None,None,None,indicative,singular,third,None,None,None,None,past,finite,active
91135,"ἴχνει_verb_{'Aspect': None, 'Case': None, 'Def...",ἴχνει,2,ἴχνος,verb,None,None,None,None,None,indicative,singular,third,None,None,None,None,present,finite,active
91138,"ὀλωλότων_verb_{'Aspect': 'perfective', 'Case':...",ὀλωλότων,2,ὄλλυμι,verb,perfective,genitive,None,None,masculine,None,plural,None,None,None,None,None,past,participle,active
91142,"διαθεῖναι_verb_{'Aspect': 'perfective', 'Case'...",διαθεῖναι,2,διατίθημι,verb,perfective,None,None,None,None,None,None,None,None,None,None,None,past,infinitive,active


In [37]:
(100 * text_counts / text_counts.sum()).sort_values(by="count", ascending=False).cumsum()

,count
text,
καὶ,5.488011
δὲ,8.262735
τῶν,9.576984
μὲν,10.872359
τὴν,12.005995
...,...
σιγῶντα,99.999552
σιγαλόεντι,99.999664
σθω,99.999776


In [39]:
(100 * lemma_counts / lemma_counts.sum()).sort_values(by="count", ascending=False).cumsum()

,count
lemma,
ὁ,12.282375
καί,17.885027
δέ,20.898048
εἰμί,22.852872
αὐτός,24.640692
...,...
μέλον,99.999552
μέλομαι,99.999664
ἀποκρύψομαι,99.999776


In [18]:
(100 * df_words["count"] / df_words["count"].sum())[:7000].sum()

78.20577368848159

In [ ]:
(100 * lemma_series / lemma_series.sum())[:6_000].sum()

In [ ]:
(100 * word_series / word_series.sum())[:6_000].sum()